In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import scipy as sc
import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
2+2

4

In [3]:
###
df = pd.read_table("/Users/cdonnat/Downloads/socfb-Berkeley13/socfb-Berkeley13.csv", sep=" ", header = None)

In [4]:
df.columns = ["From", "To"]

In [5]:
G = nx.from_pandas_edgelist(df, source='From', target='To')

In [6]:
d_max = np.asarray(nx.degree(G))[:,1].max()

In [7]:
np.quantile(np.asarray(nx.degree(G))[:,1], 0.05)

3.0

In [8]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import scipy as sc
import time

def simulate_epidemic(W, y_init, beta_v, gamma_v,
                      steps = 1, propagate = "true_p",
                      alpha_fp = 0.1, seed = 1,
                      min_clip=0):
    """
    # W : weighted graph adjacency matrix
    # y_init : initial observations
    # beta :  infection probability
    # gamma : recovery probability
    """
    np.random.seed(seed)
    n_nodes = W.shape[1]

    #### Generate ground truth:
    y = y_init
    true_p = y_init
    track_state = np.zeros((n_nodes, steps + 1))
    track_state[:, 0] = y_init
    for step in np.arange(1,steps):
        print("Step "  + str(step) + " sparsity: " + str((true_p > 0).sum() ))
        if (true_p > 0).sum() <  0.1 * n_nodes:
            print("Using sparsity")
            true_p = propagate_one_step_sparse(W, true_p, beta_v, gamma_v)
        else:
            true_p.resize((n_nodes,))
            true_p = propagate_one_step(W, true_p, beta_v, gamma_v)
        print(true_p.shape)
        track_state[:, step] = true_p
        true_p = np.reshape(true_p, (n_nodes,))
        print(step)
        true_p.resize((n_nodes,))
        true_p = np.asarray(true_p)
        print(true_p.shape)
        true_p[np.where(true_p < min_clip)[0]] = 0
        #print(true_p)
    
    y_true = np.random.binomial(n=1, p=np.clip(true_p, 0, 1))
    
    it_p = 1
    while (y_true.sum() == 0) and  it_p < 100:
      ## resample to make sure someone is infectious (we must see someone infectious)
        y_true = np.random.binomial(n=1, p=np.clip(true_p, 0, 1)) 
        it_p = it_p + 1
        print("it_p: " + str(it_p))
        
    index_observed = np.where(y_true!=0)
    #### 
    y_false = np.random.binomial(n=1, p=[alpha_fp] * n_nodes)
    y_false[index_observed] = 0
    y_observed = np.clip(y_true + y_false, 0, 1)
    

    return({"true_p" : true_p,
            "y_observed" : y_observed,
            "beta_v": beta_v,
            "gamma_v": gamma_v,
            "y_false": y_false,
            "y_true": y_true,
            "track_state": track_state
            })
    

def propagate_one_step_sparse(W, y, beta_v, gamma_v):
    """
    W : weighted graph adjacency matrix
    y_init : initial observations
    beta :  infection probability
    gamma : recovery probability
    """
    y_sp = sc.sparse.csc_matrix(y).T
    n_nodes = W.shape[0]
    D = sc.sparse.eye(n_nodes,format='csc')- sc.sparse.diags(gamma_v,format='csc') + \
        sc.sparse.diags(1 - y,format='csc') @ ( W @ sc.sparse.diags(beta_v,format='csc')) 
    true_p = D @ y_sp
    true_p = np.asarray(true_p.todense())[:,0]
    true_p = np.clip(true_p, 0, 1)
    return(true_p)


def propagate_one_step(W, y, beta_v, gamma_v):
    """
    W : weighted graph adjacency matrix
    y_init : initial observations
    beta :  infection probability
    gamma : recovery probability
    """
    W = W.todense()
    n_nodes = W.shape[0]
    D = np.eye(n_nodes)- np.diag(gamma_v) + \
        np.dot(np.diag(1 - y), np.dot( W, np.diag(beta_v)))
    true_p = np.dot(D, y.reshape((n_nodes, 1))).reshape((-1,))
    #print(D.shape, y.shape, true_p.shape)
    true_p = np.clip(true_p, 0, 1)
    return(true_p)

import time

def generate_scenario(n_nodes = 1000, beta = 0.9, gamma =0.1,
                      alpha_fp =0.001, 
                      n_init = 1, steps = 20, type_graph ="ER", p_er=0.01, 
                      p_ws = 0.1, p_pl = 0.01, m = 3,
                      seed = 1,
                      epsilon=0.01, do_plot = False,
                      min_clip=0):
    if type_graph == "ER":
        G = nx.erdos_renyi_graph(n=int(n_nodes), p=p_er, seed=seed)
    elif type_graph == "2Dgrid":
        G = nx.grid_2d_graph(n=int(np.sqrt(n_nodes)), m=int(np.sqrt(n_nodes)))
    elif type_graph == "expander":
        G = nx.paley_graph(p=n_nodes)
    elif type_graph == "small-world":
        G = nx.connected_watts_strogatz_graph(n_nodes, k=m, p=p_ws, seed = seed)
    elif type_graph == "pa":
        G = nx.barabasi_albert_graph(n_nodes, m=m, seed =seed)
    elif type_graph == "power_law":
        G = nx.powerlaw_cluster_graph(n_nodes, m=m, p_pl = p_pl, seed =seed)
    else:
        print("Type of Graph Not implemented yet")
        return()
    n_nodes = nx.number_of_nodes(G)
    d_max = np.asarray(nx.degree(G))[:,1].max()
    weights = [None] * nx.number_of_edges(G)
    it = 0
    for (u, v) in G.edges():
        G[u][v]['weight'] = 1.0/(d_max + epsilon) 
        weights[it] = 1.0/(d_max + epsilon) #random.uniform(0, 10)  # Random weight between 0 and 10
        it += 1
    W = nx.adjacency_matrix(G)
    Gamma = 1.0/(d_max + epsilon) * nx.incidence_matrix(G, oriented=True)
    
    beta_v = np.array([beta] * n_nodes)
    gamma_v = np.array([gamma] * n_nodes)
    y_init = np.zeros(n_nodes)
    ind_init = int(np.ceil( np.random.random_sample(n_init) * n_nodes))
    y_init[ind_init] = 1
    if do_plot : 
        pos = nx.kamada_kawai_layout(G)
        plt.figure()
        nx.draw(G, pos = pos, node_color = y_init,
                edge_color='gray')
        plt.show()

    
    epidemic = simulate_epidemic(W, y_init, beta_v, gamma_v,
                steps = steps, propagate = "true_p",
                alpha_fp = alpha_fp, seed=seed,
                min_clip=min_clip)
    if do_plot:
        for i in np.arange(steps + 1):
            plt.figure()
            nx.draw(G, pos = pos, node_color = epidemic["track_state"][:, i], edge_color='gray' )
            plt.show()
            time.sleep(1)
            
    return({"epidemic" : epidemic,
           "W" : W,
            "Gamma": Gamma,
            "y_init": y_init,
            "beta" : beta,
            "gamma" : gamma
           })


def generate_scenario_with_graph(G, beta = 0.9, gamma = 0.1,
                      alpha_fp =0.001, 
                      n_init = 1, steps = 20,
                      seed = 1,
                      epsilon=0.01, do_plot = False,
                      min_clip=0):
    
    n_nodes = nx.number_of_nodes(G)
    d_max = np.asarray(nx.degree(G))[:,1].max()
    weights = [None] * nx.number_of_edges(G)
    it = 0
    for (u, v) in G.edges():
        G[u][v]['weight'] = 1.0/(d_max + epsilon) 
        weights[it] = 1.0/(d_max + epsilon) #random.uniform(0, 10)  # Random weight between 0 and 10
        it += 1
    W = nx.adjacency_matrix(G)
    Gamma = 1.0/(d_max + epsilon) * nx.incidence_matrix(G, oriented=True)
    
    beta_v = np.array([beta] * n_nodes)
    gamma_v = np.array([gamma] * n_nodes)
    y_init = np.zeros(n_nodes)
    ind_init = int(np.ceil( np.random.random_sample(n_init) * n_nodes))
    y_init[ind_init] = 1
    if do_plot : 
        pos = nx.kamada_kawai_layout(G)
        plt.figure()
        nx.draw(G, pos = pos, node_color = y_init,
                edge_color='gray')
        plt.show()

    
    epidemic = simulate_epidemic(W, y_init, beta_v, gamma_v,
                steps = steps, propagate = "true_p",
                alpha_fp = alpha_fp, seed=seed,
                min_clip=min_clip)
    if do_plot:
        for i in np.arange(steps + 1):
            plt.figure()
            nx.draw(G, pos = pos, node_color = epidemic["track_state"][:, i], edge_color='gray' )
            plt.show()
            time.sleep(1)
            
    return({"epidemic" : epidemic,
            "W" : W,
            "Gamma": Gamma,
            "y_init": y_init,
            "beta" : beta,
            "gamma" : gamma
           })
    

In [9]:
import numpy as np
import pandas as pd
import time 
import numpy as np
import networkx as nx
import sys, os
sys.path.append('/Users/cdonnat/Documents/epidemic_modelling/python')

#from simulate_epidemics import * 
from solvers import *

type_graph = "ER"
steps = 30
n_step_predict = 20
n_init = 1
min_clip =1e-4
columns = ['Experiment', 'Method', 'Time', 'Lambda',  'Accuracy_true_p', 'Accuracy_true_y']
for step in np.arange(n_step_predict):
    columns += ['accuracy_prop_' + str(step), 
               'accuracy_benchmark_prop_' + str(step)]
results_df = pd.DataFrame(columns=columns)

i = 0
for exp in np.arange(100):
            ### generate epidemic
            scenario = generate_scenario_with_graph(G, beta = 0.9, gamma =0.01,
                        alpha_fp =0.00, 
                        n_init = n_init, steps = steps,
                        epsilon=0.01, do_plot = False, min_clip=1e-5)
            for lambda_ in [0.01, 0.1, 0.5, 1, 3, 5, 10, 15, 20, 30, 50, 80, 100]:
                start_time = time.time() 
                res_ssnal = ssnal_solver(scenario['epidemic']['y_observed'],scenario['W'], lambda_)
                end_time = time.time() 
                res_ssnal = res_ssnal[:,0] 
                ### Propagate solution
                temp_res = [exp, 'SSNAL', end_time - start_time,  lambda_,
                                    np.mean(np.abs(res_ssnal - scenario['epidemic']['true_p'])),
                                    np.mean(np.abs(res_ssnal  - scenario['epidemic']['y_true'])),
                                    ]
                current_p = res_ssnal
                current_p[np.where(current_p <min_clip)[0]] = 0
                current_p_observed = scenario['epidemic']['y_observed']
                ground_truth  = scenario['epidemic']['true_p']
                n_nodes = nx.number_of_nodes(G)
                beta_v = [scenario['beta']] * n_nodes
                gamma_v = [scenario['gamma']] * n_nodes
                for it in np.arange(n_step_predict):
                    print(it)
                    if (current_p > 0).sum() <  0.1 * n_nodes:
                        print("Using sparsity")
                        current_p = propagate_one_step_sparse(scenario['W'], current_p, beta_v, gamma_v)
                    else:
                        current_p.resize((n_nodes,))
                        current_p = propagate_one_step(scenario['W'], current_p, beta_v, gamma_v)
                    current_p[np.where(current_p <min_clip)[0]] = 0
                    if (current_p_observed > 0).sum() <  0.1 * n_nodes:
                        print("Using sparsity")
                        current_p_observed = propagate_one_step_sparse(scenario['W'], current_p_observed, beta_v, gamma_v)
                    else:
                        current_p_observed.resize((n_nodes,))
                        current_p_observed = propagate_one_step(scenario['W'], current_p_observed, beta_v, gamma_v)
                    current_p_observed[np.where(current_p_observed <min_clip)[0]] = 0
                    if (ground_truth > 0).sum() <  0.1 * n_nodes:
                        print("Using sparsity")
                        ground_truth = propagate_one_step_sparse(scenario['W'], ground_truth, beta_v, gamma_v)
                    else:
                        ground_truth.resize((n_nodes,))
                        ground_truth = propagate_one_step(scenario['W'], ground_truth, beta_v, gamma_v)
                    ground_truth[np.where(ground_truth <min_clip)[0]] = 0
                    temp_res += [np.mean(np.abs(current_p  - ground_truth)),
                                 np.mean(np.abs(current_p_observed  - ground_truth))]

                

                results_df.loc[i] = temp_res
                i += 1


                results_df.to_csv('~/Documents/epidemic_modelling/python/experiments/results_algo_semi_synthetic.csv', index=False)
                if np.mean(res_ssnal == 0) == 1:
                    break
                







Step 1 sparsity: 1
Using sparsity
(22900,)
1
(22900,)
Step 2 sparsity: 5
Using sparsity
(22900,)
2
(22900,)
Step 3 sparsity: 5
Using sparsity
(22900,)
3
(22900,)
Step 4 sparsity: 5
Using sparsity
(22900,)
4
(22900,)
Step 5 sparsity: 5
Using sparsity
(22900,)
5
(22900,)
Step 6 sparsity: 5
Using sparsity
(22900,)
6
(22900,)
Step 7 sparsity: 5
Using sparsity
(22900,)
7
(22900,)
Step 8 sparsity: 5
Using sparsity
(22900,)
8
(22900,)
Step 9 sparsity: 5
Using sparsity
(22900,)
9
(22900,)
Step 10 sparsity: 5
Using sparsity
(22900,)
10
(22900,)
Step 11 sparsity: 5
Using sparsity
(22900,)
11
(22900,)
Step 12 sparsity: 5
Using sparsity
(22900,)
12
(22900,)
Step 13 sparsity: 5
Using sparsity
(22900,)
13
(22900,)
Step 14 sparsity: 5
Using sparsity
(22900,)
14
(22900,)
Step 15 sparsity: 5
Using sparsity
(22900,)
15
(22900,)
Step 16 sparsity: 5
Using sparsity
(22900,)
16
(22900,)
Step 17 sparsity: 5
Using sparsity
(22900,)
17
(22900,)
Step 18 sparsity: 5
Using sparsity
(22900,)
18
(22900,)
Step 19 sp

/Users/cdonnat/Documents/pycvxcluster/src/pycvxcluster/algos/admm.py:32: CholmodTypeConversionWarning: converting matrix of class csr_matrix to CSC format
  factor = cholesky(eye(n) + sigma * A @ A.T)
/Users/cdonnat/Documents/pycvxcluster/src/pycvxcluster/algos/ssnal.py:339: RuntimeWarning: invalid value encountered in double_scalars
  dual_ratio = dualinf_sub / dualinf_hist[itersub - 1]
/Users/cdonnat/Documents/pycvxcluster/src/pycvxcluster/algos/ssnal.py:438: RuntimeWarning: invalid value encountered in double_scalars
  dualinf_ratio = dualinf_hist[itersub] / dualinf_hist[itersub - 1]
/Users/cdonnat/Documents/pycvxcluster/src/pycvxcluster/algos/ssnal.py:477: RuntimeWarning: invalid value encountered in divide
  dualinf_ratioall = dualinf_hist[1 : itersub + 1] / dualinf_hist[:itersub]


0
Using sparsity
Using sparsity
Using sparsity
1
Using sparsity
Using sparsity
Using sparsity
2
Using sparsity
Using sparsity
Using sparsity
3
Using sparsity
Using sparsity
Using sparsity
4
Using sparsity
Using sparsity
Using sparsity
5
Using sparsity
Using sparsity
Using sparsity
6
Using sparsity
Using sparsity
Using sparsity
7
Using sparsity
Using sparsity
Using sparsity
8
Using sparsity
Using sparsity
Using sparsity
9
Using sparsity
Using sparsity
Using sparsity
10
Using sparsity
Using sparsity
Using sparsity
11
Using sparsity
Using sparsity
Using sparsity
12
Using sparsity
Using sparsity
Using sparsity
13
Using sparsity
Using sparsity
Using sparsity
14
Using sparsity
Using sparsity
Using sparsity
15
Using sparsity
Using sparsity
Using sparsity
16
Using sparsity
Using sparsity
Using sparsity
17
Using sparsity
Using sparsity
Using sparsity
18
Using sparsity
Using sparsity
Using sparsity
19
Using sparsity
Using sparsity
Using sparsity


/Users/cdonnat/Documents/pycvxcluster/src/pycvxcluster/algos/admm.py:32: CholmodTypeConversionWarning: converting matrix of class csr_matrix to CSC format
  factor = cholesky(eye(n) + sigma * A @ A.T)
/Users/cdonnat/Documents/pycvxcluster/src/pycvxcluster/algos/ssnal.py:339: RuntimeWarning: invalid value encountered in double_scalars
  dual_ratio = dualinf_sub / dualinf_hist[itersub - 1]
/Users/cdonnat/Documents/pycvxcluster/src/pycvxcluster/algos/ssnal.py:438: RuntimeWarning: invalid value encountered in double_scalars
  dualinf_ratio = dualinf_hist[itersub] / dualinf_hist[itersub - 1]


0
Using sparsity
Using sparsity
Using sparsity
1
Using sparsity
Using sparsity
Using sparsity
2
Using sparsity
Using sparsity
Using sparsity
3
Using sparsity
Using sparsity
Using sparsity
4
Using sparsity
Using sparsity
Using sparsity
5
Using sparsity
Using sparsity
Using sparsity
6
Using sparsity
Using sparsity
Using sparsity
7
Using sparsity
Using sparsity
Using sparsity
8
Using sparsity
Using sparsity
Using sparsity
9
Using sparsity
Using sparsity
Using sparsity
10
Using sparsity
Using sparsity
Using sparsity
11
Using sparsity
Using sparsity
Using sparsity
12
Using sparsity
Using sparsity
Using sparsity
13
Using sparsity
Using sparsity
Using sparsity
14
Using sparsity
Using sparsity
Using sparsity
15
Using sparsity
Using sparsity
Using sparsity
16
Using sparsity
Using sparsity
Using sparsity
17
Using sparsity
Using sparsity
Using sparsity
18
Using sparsity
Using sparsity
Using sparsity
19
Using sparsity
Using sparsity
Using sparsity


...

/Users/cdonnat/Documents/pycvxcluster/src/pycvxcluster/algos/admm.py:32: CholmodTypeConversionWarning: converting matrix of class csr_matrix to CSC format
  factor = cholesky(eye(n) + sigma * A @ A.T)
/Users/cdonnat/Documents/pycvxcluster/src/pycvxcluster/algos/ssnal.py:339: RuntimeWarning: invalid value encountered in double_scalars
  dual_ratio = dualinf_sub / dualinf_hist[itersub - 1]
/Users/cdonnat/Documents/pycvxcluster/src/pycvxcluster/algos/ssnal.py:438: RuntimeWarning: invalid value encountered in double_scalars
  dualinf_ratio = dualinf_hist[itersub] / dualinf_hist[itersub - 1]


0
Using sparsity
Using sparsity
Using sparsity
1
Using sparsity
Using sparsity
Using sparsity
2
Using sparsity
Using sparsity
Using sparsity
3
Using sparsity
Using sparsity
Using sparsity
4
Using sparsity
Using sparsity
Using sparsity
5
Using sparsity
Using sparsity
Using sparsity
6
Using sparsity
Using sparsity
Using sparsity
7
Using sparsity
Using sparsity
Using sparsity
8
Using sparsity
Using sparsity
Using sparsity
9
Using sparsity
Using sparsity
Using sparsity
10
Using sparsity
Using sparsity
Using sparsity
11
Using sparsity
Using sparsity
Using sparsity
12
Using sparsity
Using sparsity
Using sparsity
13
Using sparsity
Using sparsity
Using sparsity
14
Using sparsity
Using sparsity
Using sparsity
15
Using sparsity
Using sparsity
Using sparsity
16
Using sparsity
Using sparsity
Using sparsity
17
Using sparsity
Using sparsity
Using sparsity
18
Using sparsity
Using sparsity
Using sparsity
19
Using sparsity
Using sparsity
Using sparsity


/Users/cdonnat/Documents/pycvxcluster/src/pycvxcluster/algos/admm.py:32: CholmodTypeConversionWarning: converting matrix of class csr_matrix to CSC format
  factor = cholesky(eye(n) + sigma * A @ A.T)
/Users/cdonnat/Documents/pycvxcluster/src/pycvxcluster/algos/ssnal.py:339: RuntimeWarning: invalid value encountered in double_scalars
  dual_ratio = dualinf_sub / dualinf_hist[itersub - 1]
/Users/cdonnat/Documents/pycvxcluster/src/pycvxcluster/algos/ssnal.py:438: RuntimeWarning: invalid value encountered in double_scalars
  dualinf_ratio = dualinf_hist[itersub] / dualinf_hist[itersub - 1]


0
Using sparsity
Using sparsity
Using sparsity
1
Using sparsity
Using sparsity
Using sparsity
2
Using sparsity
Using sparsity
Using sparsity
3
Using sparsity
Using sparsity
Using sparsity
4
Using sparsity
Using sparsity
Using sparsity
5
Using sparsity
Using sparsity
Using sparsity
6
Using sparsity
Using sparsity
Using sparsity
7
Using sparsity
Using sparsity
Using sparsity
8
Using sparsity
Using sparsity
Using sparsity
9
Using sparsity
Using sparsity
Using sparsity
10
Using sparsity
Using sparsity
Using sparsity
11
Using sparsity
Using sparsity
Using sparsity
12
Using sparsity
Using sparsity
Using sparsity
13
Using sparsity
Using sparsity
Using sparsity
14
Using sparsity
Using sparsity
Using sparsity
15
Using sparsity
Using sparsity
Using sparsity
16
Using sparsity
Using sparsity
Using sparsity
17
Using sparsity
Using sparsity
Using sparsity
18
Using sparsity
Using sparsity
Using sparsity
19
Using sparsity
Using sparsity
Using sparsity


/Users/cdonnat/Documents/pycvxcluster/src/pycvxcluster/algos/admm.py:32: CholmodTypeConversionWarning: converting matrix of class csr_matrix to CSC format
  factor = cholesky(eye(n) + sigma * A @ A.T)

KeyboardInterrupt



In [ ]:
results_df

In [20]:
columns = ['Experiment', 'Method', 'Time', 'Lambda',  'Accuracy_true_p', 'Accuracy_true_y']
for step in np.arange(20):
    columns += ['accuracy_prop_' + str(step), 
               'accuracy_benchmark_prop_' + str(step)]

In [21]:
results_df = pd.DataFrame(columns=columns)


In [22]:
                    ground_truth[np.where(ground_truth <min_clip)[0]] = 0
                    temp_res += [np.mean(np.abs(current_p  - ground_truth)),
                                 np.mean(np.abs(current_p_observed  - ground_truth))]

In [27]:
res_ssnal[res_ssnal>0]


array([0.99993593])

In [15]:
res_temp

NameError: name 'res_temp' is not defined

In [ ]:
                for it in np.arange(20):
                    print(it)
                    if (current_p > 0).sum() <  0.1 * n_nodes:
                        print("Using sparsity")
                        current_p = propagate_one_step_sparse(scenario['W'], current_p, beta_v, gamma_v)
                    else:
                        current_p.resize((n_nodes,))
                        current_p = propagate_one_step(scenario['W'], current_p, beta_v, gamma_v)
                    current_p[np.where(current_p <min_clip)[0]] = 0
                    if (current_p_observed > 0).sum() <  0.1 * n_nodes:
                        print("Using sparsity")
                        current_p_observed = propagate_one_step_sparse(scenario['W'], current_p_observed, beta_v, gamma_v)
                    else:
                        current_p_observed.resize((n_nodes,))
                        current_p_observed = propagate_one_step(scenario['W'], current_p_observed, beta_v, gamma_v)
                    current_p_observed[np.where(current_p_observed <min_clip)[0]] = 0
                    if (ground_truth > 0).sum() <  0.1 * n_nodes:
                        print("Using sparsity")
                        ground_truth = propagate_one_step_sparse(scenario['W'], ground_truth, beta_v, gamma_v)
                    else:
                        ground_truth.resize((n_nodes,))
                        ground_truth = propagate_one_step(scenario['W'], ground_truth, beta_v, gamma_v)
                    ground_truth[np.where(ground_truths <min_clip)[0]] = 0
                    temp_res += [np.mean(np.abs(current_p  - ground_truth)),
                                 np.mean(np.abs(current_p_observed  - ground_truth))]

In [ ]:
current_p[np.where(current_p <1e-5)[0]] = 0

In [ ]:
                current_p = res_ssnal
                current_p_observed = scenario['epidemic']['y_observed']
                ground_truth  = scenario['epidemic']['true_p']

In [ ]:
current_p.max()

In [ ]:
                n_nodes = nx.number_of_nodes(G)
                beta_v = [scenario['beta']] * n_nodes
                gamma_v = [scenario['gamma']] * n_nodes

In [ ]:
                for it in np.arange(20):
                    print(it)
                    if (current_p > 0).sum() <  0.1 * n_nodes:
                        print("Using sparsity")
                        current_p = propagate_one_step_sparse(scenario['W'], current_p, beta_v, gamma_v)
                    else:
                        current_p.resize((n_nodes,))
                        current_p = propagate_one_step(scenario['W'], current_p, beta_v, gamma_v)
                    if (current_p_observed > 0).sum() <  0.1 * n_nodes:
                        print("Using sparsity")
                        current_p_observed = propagate_one_step_sparse(scenario['W'], current_p_observed, beta_v, gamma_v)
                    else:
                        current_p_observed.resize((n_nodes,))
                        current_p_observed = propagate_one_step(scenario['W'], current_p_observed, beta_v, gamma_v)
                    if (ground_truth > 0).sum() <  0.1 * n_nodes:
                        print("Using sparsity")
                        ground_truth = propagate_one_step_sparse(scenario['W'], ground_truth, beta_v, gamma_v)
                    else:
                        ground_truth.resize((n_nodes,))
                        ground_truth = propagate_one_step(scenario['W'], ground_truth, beta_v, gamma_v)
                    temp_res += [np.mean(np.abs(current_p  - ground_truth)),
                                 np.mean(np.abs(current_p_observed  - ground_truth))]

In [ ]:
start_time = time.time() 
res_cvx = cvx_solver(scenario['epidemic']['y_observed'], scenario['Gamma'].todense().T, lambda_)
end_time = time.time() 
results_df.loc[i] = [exp, 'CVX', end_time - start_time,  lambda_, n_nodes, p_er, 
                                    np.mean(np.abs(res_cvx  - scenario['epidemic']['true_p'])),
                                    np.mean(np.abs(res_cvx  - scenario['epidemic']['y_true'])) ]

In [ ]:
n_nodes = nx.number_of_nodes(G)

In [ ]:
lambda_ = 100

In [ ]:
                start_time = time.time() 
                res_ssnal = ssnal_solver(scenario['epidemic']['y_observed'],scenario['W'], lambda_)
                end_time = time.time() 
                res_ssnal = res_ssnal[:,0]
                results_df.loc[i] = [exp, 'SSNAL', end_time - start_time,  lambda_, n_nodes, 0, 
                                    np.mean(np.abs(res_ssnal - scenario['epidemic']['true_p'])),
                                    np.mean(np.abs(res_ssnal  - scenario['epidemic']['y_true']))]

In [ ]:
np.mean(np.abs(res_ssnal  - scenario['epidemic']['true_p'])), np.mean(np.abs(scenario['epidemic']['y_observed'] - scenario['epidemic']['true_p']))

In [ ]:
    n_nodes = nx.number_of_nodes(G)
    d_max = np.asarray(nx.degree(G))[:,1].max()
    weights = [None] * nx.number_of_edges(G)
    it = 0
    for (u, v) in G.edges():
        G[u][v]['weight'] = 1.0/(d_max + epsilon) 
        weights[it] = 1.0/(d_max + epsilon) #random.uniform(0, 10)  # Random weight between 0 and 10
        it += 1
    W = nx.adjacency_matrix(G)
    Gamma = 1.0/(d_max + epsilon) * nx.incidence_matrix(G, oriented=True)

In [ ]:
    beta_v = np.array([beta] * n_nodes)
    gamma_v = np.array([gamma] * n_nodes)
    y_init = np.zeros(n_nodes)
    ind_init = int(np.ceil( np.random.random_sample(n_init) * n_nodes))
    y_init[ind_init] = 1

In [ ]:
    np.random.seed(1)
    n_nodes = W.shape[1]

    #### Generate ground truth:
    y = y_init
    true_p = y_init
    track_state = np.zeros((n_nodes, steps + 1))
    track_state[:, 0] = y_init
    for step in np.arange(1, steps+1):
        STOP
        print("Step "  + str(step) + " sparsity: " + str((true_p > 0).sum() ))
        if (true_p > 0).sum() <  0.1 * n_nodes:
            print("Using sparsity")
            true_p = propagate_one_step_sparse(W, true_p, beta_v, gamma_v,
                                               min_clip = min_clip, max_clip=max_clip)
        else:
            true_p.resize((n_nodes,))
            true_p = propagate_one_step(W, true_p, beta_v, gamma_v,
                                        min_clip = min_clip, max_clip=max_clip)
        print(true_p.shape)
        track_state[:, step] = true_p
        true_p = np.reshape(true_p, (n_nodes,))
        print(step)
        true_p.resize((n_nodes,))
        true_p = np.asarray(true_p)
        print(true_p.shape)
        #print(true_p)

In [ ]:
min_clip = 1e-5
max_clip=1

In [ ]:
true_p = propagate_one_step_sparse(W, true_p, beta_v, gamma_v,
                                   min_clip = min_clip, max_clip=max_clip)

In [ ]:
true_p